In [1]:
import pandas as pd

In [ ]:

one = pd.read_csv('one_rows.csv', header=None)
zero = pd.read_csv('zero_rows.csv', header=None)

print(one.shape)
print(zero.shape)
# 把zero随机删除951行
zero = zero.drop(zero.sample(n=951).index)
zero.reset_index(drop=True, inplace=True)
print(zero.shape)
# save csv
zero.to_csv('zero_rows_del.csv', index=False, header=False)
print('zero')
# 从zero随机取2150行
zero_t = zero.sample(n=2150)
# 获取这些抽取行的索引
zero_t_indices = zero_t.index
# 删除这些行
zero_v = zero.drop(zero_t_indices)
print('one')
# 从one随机取430行
one_t = one.sample(n=2150)
# 获取这些抽取行的索引
one_t_indices = one_t.index
# 删除这些行
one_v = one.drop(one_t_indices)
print('combine_t')
# 交叉合并 zero_t 和 one_t
# 创建一个新的 DataFrame，用于交叉合并 zero_t 和 one_t
combined_t = pd.DataFrame()

# 交替添加 zero_t 和 one_t 的行
for i in range(2150):
    # print(i)
    combined_t = pd.concat([combined_t, zero_t.iloc[[i]]])
    combined_t = pd.concat([combined_t, one_t.iloc[[i]]])

# 重置索引
combined_t = combined_t.reset_index(drop=True)
combined_t.to_csv('train.csv', index=False, header=False)
print(combined_t.shape)

print('combine_v')
combined_v = pd.DataFrame()

# 交替添加 zero_t 和 one_t 的行
for i in range(430):
    combined_v = pd.concat([combined_v, zero_v.iloc[[i]]])
    combined_v = pd.concat([combined_v, one_v.iloc[[i]]])

# 重置索引
combined_v = combined_v.reset_index(drop=True)
combined_v.to_csv('valid.csv', index=False, header=False)
print(combined_v.shape)


In [33]:
one = pd.read_csv('one_rows.csv', header=None)
zero = pd.read_csv('zero_rows.csv', header=None)

print(one[1].apply(str).apply(len).describe())
print(zero[1].apply(str).apply(len).describe())


count     2580.000000
mean      2771.193023
std       2241.240118
min        133.000000
25%       1254.750000
50%       2086.000000
75%       3706.000000
max      20688.000000
Name: 1, dtype: float64
count     3531.000000
mean      4815.411781
std       3124.972060
min        139.000000
25%       2924.000000
50%       4093.000000
75%       5745.500000
max      46398.000000
Name: 1, dtype: float64
